# Request for Proposal for various Insurance Policies for India Postal Payments Bank Document Summarization

## Importing Relevant Libraries

In [1]:
from __future__ import print_function

#Relevant Libraries for Loading Data
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

#Relevant Analysis Librariess
import re
import numpy as np
import pandas as pd
import mglearn as mg

#Relevant Modeling Libraries
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

%pylab
%matplotlib inline

#Python LDA Visualization Libraries
import pyLDAvis
import pyLDAvis.sklearn

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


C:\Users\DELL\Anaconda3\lib\site-packages\past\types\oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
C:\Users\DELL\Anaconda3\lib\site-packages\past\builtins\misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping


## Loading Data

In [2]:
#Creating a Function to Read the PDF Document and convert into Text
def convert_pdf_to_text(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()
    
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password, caching=caching, check_extractable=True):
        interpreter.process_page(page)
        
    text = retstr.getvalue()
    
    fp.close()
    device.close()
    retstr.close()
    
    return text

#Reading the PDF Document and saving as lone
lone = convert_pdf_to_text('C:\\Users\\DELL\\Desktop\\Akshay\\Projects\\rfp_ipp_bank_document_summarization\\rfp_ipp_bank_document.pdf')

## 1. Data Exploration

In [3]:
#Exploring the complete document
lone

" \n \n \n \n\nREQUEST FOR PROPOSAL (RFP) \n\nfor \n\nvarious Insurance Policies  \n\nfor  \n\nIndia Post Payments Bank Limited \n\nDate: May 14, 2019 \n\nIndia Post Payments Bank \n\n  \n\n \n\n\x0cInvitation for Request for Proposal (RFP) \n\nIPPB invites proposals from eligible general insurance companies for various insurance requirements \nof India Post Payments Bank (IPPB). \n\nThis  RFP  may  be  downloaded  by \n(www.eprocure.gov.in). \n\nthe  bidders  free  of  cost  from  the  e-procure  website \n\nRFP Summary Sheet \n\nName of the company \nProcurement Reference Number \nTender Cost \nDate of issue of the RFP \nPre-bid meeting date \nLast date to seek clarification \nLast Date for reply of queries \nDate and Time for RFP submission / upload of \nbids \nDate and time for opening of RFP \nPrimary point of contact for RFP process  and \ntechnical queries \n\nInsurance Broker Contact Details \n\nPlace of Pre-bid meeting \n\nPlace of Bid submission and RFP Opening \n\nAddress fo

## 2. Data Preprocessing

In [4]:
#Splitting lines
clean_cont = lone.splitlines()

#Exploring the document after splitting lines
clean_cont

[' ',
 ' ',
 ' ',
 ' ',
 '',
 'REQUEST FOR PROPOSAL (RFP) ',
 '',
 'for ',
 '',
 'various Insurance Policies  ',
 '',
 'for  ',
 '',
 'India Post Payments Bank Limited ',
 '',
 'Date: May 14, 2019 ',
 '',
 'India Post Payments Bank ',
 '',
 '  ',
 '',
 ' ',
 '',
 '',
 'Invitation for Request for Proposal (RFP) ',
 '',
 'IPPB invites proposals from eligible general insurance companies for various insurance requirements ',
 'of India Post Payments Bank (IPPB). ',
 '',
 'This  RFP  may  be  downloaded  by ',
 '(www.eprocure.gov.in). ',
 '',
 'the  bidders  free  of  cost  from  the  e-procure  website ',
 '',
 'RFP Summary Sheet ',
 '',
 'Name of the company ',
 'Procurement Reference Number ',
 'Tender Cost ',
 'Date of issue of the RFP ',
 'Pre-bid meeting date ',
 'Last date to seek clarification ',
 'Last Date for reply of queries ',
 'Date and Time for RFP submission / upload of ',
 'bids ',
 'Date and time for opening of RFP ',
 'Primary point of contact for RFP process  and ',
 'te

In [5]:
#Removing all Non-Alphabetic Characters from the document
shear = [re.sub("[^a-zA-Z]+", " ", s) for s in clean_cont]

#Removing the unnecessary spaces from the document
shears = [x for x in shear if x != ' ']
shearss = [x for x in shears if x != '']

In [6]:
shear

[' ',
 ' ',
 ' ',
 ' ',
 '',
 'REQUEST FOR PROPOSAL RFP ',
 '',
 'for ',
 '',
 'various Insurance Policies ',
 '',
 'for ',
 '',
 'India Post Payments Bank Limited ',
 '',
 'Date May ',
 '',
 'India Post Payments Bank ',
 '',
 ' ',
 '',
 ' ',
 '',
 '',
 'Invitation for Request for Proposal RFP ',
 '',
 'IPPB invites proposals from eligible general insurance companies for various insurance requirements ',
 'of India Post Payments Bank IPPB ',
 '',
 'This RFP may be downloaded by ',
 ' www eprocure gov in ',
 '',
 'the bidders free of cost from the e procure website ',
 '',
 'RFP Summary Sheet ',
 '',
 'Name of the company ',
 'Procurement Reference Number ',
 'Tender Cost ',
 'Date of issue of the RFP ',
 'Pre bid meeting date ',
 'Last date to seek clarification ',
 'Last Date for reply of queries ',
 'Date and Time for RFP submission upload of ',
 'bids ',
 'Date and time for opening of RFP ',
 'Primary point of contact for RFP process and ',
 'technical queries ',
 '',
 'Insurance Br

In [7]:
shears

['',
 'REQUEST FOR PROPOSAL RFP ',
 '',
 'for ',
 '',
 'various Insurance Policies ',
 '',
 'for ',
 '',
 'India Post Payments Bank Limited ',
 '',
 'Date May ',
 '',
 'India Post Payments Bank ',
 '',
 '',
 '',
 '',
 'Invitation for Request for Proposal RFP ',
 '',
 'IPPB invites proposals from eligible general insurance companies for various insurance requirements ',
 'of India Post Payments Bank IPPB ',
 '',
 'This RFP may be downloaded by ',
 ' www eprocure gov in ',
 '',
 'the bidders free of cost from the e procure website ',
 '',
 'RFP Summary Sheet ',
 '',
 'Name of the company ',
 'Procurement Reference Number ',
 'Tender Cost ',
 'Date of issue of the RFP ',
 'Pre bid meeting date ',
 'Last date to seek clarification ',
 'Last Date for reply of queries ',
 'Date and Time for RFP submission upload of ',
 'bids ',
 'Date and time for opening of RFP ',
 'Primary point of contact for RFP process and ',
 'technical queries ',
 '',
 'Insurance Broker Contact Details ',
 '',
 'Place

In [8]:
shearss

['REQUEST FOR PROPOSAL RFP ',
 'for ',
 'various Insurance Policies ',
 'for ',
 'India Post Payments Bank Limited ',
 'Date May ',
 'India Post Payments Bank ',
 'Invitation for Request for Proposal RFP ',
 'IPPB invites proposals from eligible general insurance companies for various insurance requirements ',
 'of India Post Payments Bank IPPB ',
 'This RFP may be downloaded by ',
 ' www eprocure gov in ',
 'the bidders free of cost from the e procure website ',
 'RFP Summary Sheet ',
 'Name of the company ',
 'Procurement Reference Number ',
 'Tender Cost ',
 'Date of issue of the RFP ',
 'Pre bid meeting date ',
 'Last date to seek clarification ',
 'Last Date for reply of queries ',
 'Date and Time for RFP submission upload of ',
 'bids ',
 'Date and time for opening of RFP ',
 'Primary point of contact for RFP process and ',
 'technical queries ',
 'Insurance Broker Contact Details ',
 'Place of Pre bid meeting ',
 'Place of Bid submission and RFP Opening ',
 'Address for Communic

## 3. Topic Modeling

In [9]:
#Fitting Count Vectorizer on the document with Stop Words
vect = CountVectorizer(ngram_range=(1,1), stop_words='english')
dtm = vect.fit_transform(shearss)

#Document Term Matrix
dtm

<1609x1491 sparse matrix of type '<class 'numpy.int64'>'
	with 5610 stored elements in Compressed Sparse Row format>

In [10]:
#Converting the Document Term Matrix from Count Vectorizer into a Pandas Dataframe
pd.DataFrame(dtm.toarray(), columns=vect.get_feature_names())

,abide,ability,absolute,accept,acceptable,acceptance,accepted,access,accessed,accidental,...,writing,writings,written,wrong,wrongful,www,year,years,yes,yyyy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1604,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1605,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1606,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1607,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [11]:
#Fitting the Latent Dirichlet Allocation Model on the Document Term Matrix
lda = LatentDirichletAllocation(n_components=5)
lda_dtf = lda.fit_transform(dtm)

#Latent Dirichlet Allocation Model
lda_dtf

array([[0.05043533, 0.05118097, 0.05008478, 0.79822669, 0.05007223],
       [0.2       , 0.2       , 0.2       , 0.2       , 0.2       ],
       [0.05005459, 0.79686244, 0.05000192, 0.05014291, 0.05293814],
       ...,
       [0.04000383, 0.04020026, 0.83978778, 0.04000399, 0.04000414],
       [0.24436839, 0.02222439, 0.68866528, 0.02224481, 0.02249713],
       [0.05005707, 0.7968692 , 0.05000529, 0.05000396, 0.0530645 ]])

## 4. Topic Extracting

In [12]:
#Extracting 5 Topics from LDA and the most common words in each topic
sorting = np.argsort(lda.components_)[:, ::-1]
features = np.array(vect.get_feature_names())

mg.tools.print_topics(topics=range(5), feature_names=features, sorting=sorting, topics_per_chunk=5, n_words=15)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
bank          bid           inr           insurer       insurance     
post          annexure      loss          ippb          including     
india         rfp           attorney      limit         limits        
insured       terms         subject       assets        bidder        
date          response      new           submitted     costs         
provided      conditions    power         document      clause        
payments      marsh         delhi         cover         company       
policy        bidder        bidder        insured       data          
shall         policy        clause        bidder        information   
bidders       insurance     minimum       technical     documents     
signatory     commercial    claim         payment       years         
sum           ippb          cover         employees     equipment     
period

In [13]:
#Senteces within the Topic Model 1
topic_0 = np.argsort(lda_dtf[:,0])[::-1]
for i in topic_0[:5]:
    print(f".".join(shearss[i].split(f".")[:2]) + f".\n")

the Banking Regulation Act duly licensed by Reserve Bank of India IPPB is engaged in .

India Post Payments Bank Ltd nd Floor Speed Post Centre .

India Post Payments Bank Ltd nd Floor Speed Post Centre .

India Post Payments Bank Ltd nd Floor Speed Post Centre .

India Post Payments Bank Ltd nd Floor Speed Post Centre .



In [14]:
#Senteces within the Topic Model 2
topic_1 = np.argsort(lda_dtf[:,1])[::-1]
for i in topic_1[:5]:
    print(f".".join(shearss[i].split(f".")[:2]) + f".\n")

Corporate Office nd Floor Speed Post Centre Building Bhai Veer Singh Marg Gole Market .

Corporate Office nd Floor Speed Post Centre Building Bhai Veer Singh Marg Gole Market .

Corporate Office nd Floor Speed Post Centre Building Bhai Veer Singh Marg Gole Market .

Corporate Office nd Floor Speed Post Centre Building Bhai Veer Singh Marg Gole Market .

temporary offices currency chests vaults Post offices BO SO HO customer access points etc .



In [15]:
#Senteces within the Topic Model 3
topic_2 = np.argsort(lda_dtf[:,2])[::-1]
for i in topic_2[:5]:
    print(f".".join(shearss[i].split(f".")[:2]) + f".\n")

Certificate DSC Class II or Class III Certificates with signing key usage issued by any .

Laptops Mobiles Cell Phone POS Transaction Processing Equipment I Pad Tablet .

Building Bhai Veer Singh Marg Gole Market New Delhi .

Building Bhai Veer Singh Marg Gole Market New Delhi .

Building Bhai Veer Singh Marg Gole Market New Delhi .



In [16]:
#Senteces within the Topic Model 4
topic_3 = np.argsort(lda_dtf[:,3])[::-1]
for i in topic_3[:5]:
    print(f".".join(shearss[i].split(f".")[:2]) + f".\n")

online Bids on the CPP Portal may be obtained at https eprocure gov in eprocure app .

https eprocure gov in eprocure app by clicking on the link Online Bidder Enrolment .

 k IPPB s right to vary Marsh in consultation with IPPB reserves the right to vary any aspect of .

bidding process comprising of Technical Bid and Financial Bid from IRDAI Licensed General .

assets or financial condition or Marsh IPPB s reputation or render the Bidder unable to .



In [17]:
#Senteces within the Topic Model 5
topic_4 = np.argsort(lda_dtf[:,4])[::-1]
for i in topic_4[:5]:
    print(f".".join(shearss[i].split(f".")[:2]) + f".\n")

 Covers required Cash in safe cash in transit including Burglary Theft Dacoity .

RFP document including but not limited to proposed covers additional covers limits of .

 Central Processing Centre Delhi FFF Electrical Installations Computers and other assets .

All types of Hardware and Networking equipment Servers data processing equipment .

 Demonstrate the ability to deliver high quality services at a competitive price .



## 5. Topic Visualization

In [18]:
zit=pyLDAvis.sklearn.prepare(lda,dtm,vect)

pyLDAvis.display(zit)

C:\Users\DELL\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
